In [265]:
import numpy as np
import pandas as pd
import json
import matplotlib.cm as cm
import matplotlib.colors as colors

# Question 1

In [266]:
#tdata = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

### Investigate the data

In [267]:
type(tdata)
print(len(tdata), len(tdata[0]))

3 180


In [268]:
for tbl in tdata:
    print(tbl[0:5])

  Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
                                          Unnamed: 0 Canadian postal codes  \
0  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...                    NL   
1                                                 NL                    NS   
2                                                  A                     B   

  Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  \
0         NS         PE         NB         QC         ON         MB   
1         PE         NB         QC         ON         MB         SK   
2          C          E          G          H          J          K   

  Unnamed: 8 Unnamed: 9     ...     Unnamed: 21 

### We need the first table.  Convert it into DataFrame
#### Remove space from column names

In [269]:
neighs = pd.DataFrame(tdata[0])
neighs.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
print(neighs.shape)
print(neighs.columns)
neighs.head()

(180, 3)
Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Accept only rows with Borough != 'Not assigned'
### Convert all Neighborhood == 'Not assigned' into Neighborhood=Borough

In [270]:
neighs = neighs[neighs.Borough != 'Not assigned']
neighs.loc[(neighs.Neighborhood=='Not assigned'), 'Neighborhood']=neighs.Borough
neighs.reset_index(drop=True, inplace=True)
print(neighs.shape)
neighs.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [271]:
#neighs.groupby('PostalCode')
print(neighs.shape)
neighs.head(12)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [272]:
print(neighs.shape)

(103, 3)


# Question 2

In [273]:
#import geocoder # import geocoder
#postal_code = 'M5G'
# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#-----------The above section is Not Working---------------------------

#latlons = pd.read_csv('https://cocl.us/Geospatial_data')
latlons.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
print(latlons.shape)
latlons.head()

neighlocs = pd.merge(neighs, latlons, on='PostalCode', how='outer')
print(neighlocs.head())
print("\n***shape of neighlocs: ", neighlocs.shape, '***')

(103, 3)
  PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  

***shape of neighlocs:  (103, 5) ***


# Question 3

### Get data and check by plotting on a map

In [274]:
import folium
import requests
central_lat =  43.741667
central_lon = -79.373333
map_toro = folium.Map(location=[central_lat, central_lon], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(neighlocs['Latitude'], neighlocs['Longitude'], neighlocs['Borough'], neighlocs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toro)  
    
map_toro

### Initialise Forsquare credentials

In [275]:
CLIENT_ID = 'ZO4QY0P0RAH4CMHYEAE20ACVK1JXHSRMRJDOBJKTTAP5BDK4' # your Foursquare ID
CLIENT_SECRET = 'CNB0GLBLRBEI5JGLWGCRSJB3TYDJ3EAVR2N33ECPA4YOT3UR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

### Get nearby venues for each Neighborhood

In [276]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [277]:
##### function to get nearby venues for each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


#testv = getNearbyVenues( names=neighlocs.iloc[74:75]['Neighborhood'], latitudes=neighlocs.iloc[74:75]['Latitude'], longitudes=neighlocs.iloc[74:75]['Longitude'])
#print(testv)

In [279]:
toro_venues = getNearbyVenues(names=neighlocs['Neighborhood'],
                                   latitudes=neighlocs['Latitude'],
                                   longitudes=neighlocs['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

### Check shape, structure and statistics of venues data frame

In [280]:
print(toro_venues.shape)
toro_venues.head()

(1327, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [282]:
toro_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()

,Venue
Neighborhood,
Agincourt,5
"Alderwood, Long Branch",6
"Bathurst Manor, Wilson Heights, Downsview North",21
Bayview Village,4
"Bedford Park, Lawrence Manor East",24
Berczy Park,30
"Birch Cliff, Cliffside West",4
"Brockton, Parkdale Village, Exhibition Place",24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17


In [283]:
print( "#unique categories: {}".format( len(toro_venues['Venue Category'].unique()) ) )

#unique categories: 231


### Analyze neighborhoods

In [284]:
# one hot encoding
toro_onehot = pd.get_dummies(toro_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toro_onehot['Neighborhood'] = toro_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toro_onehot.columns[-1]] + list(toro_onehot.columns[:-1])
toro_onehot = toro_onehot[fixed_columns]

toro_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [285]:
#One hot encoding
toro_grouped = toro_onehot.groupby('Neighborhood').mean().reset_index()
print(toro_onehot.shape, toro_grouped.shape)
toro_grouped.head()

(1327, 231) (94, 231)


,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Generate list of top 10 venues for each Neighborhood


In [286]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [287]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toro_grouped['Neighborhood']

for ind in np.arange(toro_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Women's Store,Department Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pub,Sandwich Place,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gift Shop,Mobile Phone Shop,Shopping Mall,Bridal Shop,Sandwich Place,Deli / Bodega,Restaurant,Supermarket
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Grocery Store,Thai Restaurant,Juice Bar,Butcher,Café,Indian Restaurant


### Clustering

In [288]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toro_grouped_clustering = toro_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toro_grouped_clustering)


In [289]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toro_merged = neighlocs
# merge toronto_grouped with neighlocs to add latitude/longitude for each neighborhood
# perform inner joint to avoid missing values
toro_merged = toro_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')
toro_merged.reset_index(inplace=True)
toro_merged.head() # check the last columns!
print(toro_merged.shape, neighlocs.shape, neighborhoods_venues_sorted.shape)
print(toro_merged.head(10))

(98, 17) (103, 5) (94, 12)
   index PostalCode           Borough  \
0      0        M3A        North York   
1      1        M4A        North York   
2      2        M5A  Downtown Toronto   
3      3        M6A        North York   
4      4        M7A  Downtown Toronto   
5      6        M1B       Scarborough   
6      7        M3B        North York   
7     13        M3C        North York   
8      8        M4B         East York   
9      9        M5B  Downtown Toronto   

                                  Neighborhood   Latitude  Longitude  \
0                                    Parkwoods  43.753259 -79.329656   
1                             Victoria Village  43.725882 -79.315572   
2                    Regent Park, Harbourfront  43.654260 -79.360636   
3             Lawrence Manor, Lawrence Heights  43.718518 -79.464763   
4  Queen's Park, Ontario Provincial Government  43.662301 -79.389494   
5                               Malvern, Rouge  43.806686 -79.194353   
6                

### Final Map

In [290]:
# create map
map_clusters = folium.Map(location=[central_lat, central_lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toro_merged['Latitude'], toro_merged['Longitude'], toro_merged['Neighborhood'], toro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters